In [4]:
# merge counts

# load reference
Pep30lib_file = "/home/xw2629/noncoding_translation/old-data/oligo-pool-design/XuebingWu-12K-120-oligo-pool.fa.translated.txt"
Pep30lib=read.csv(Pep30lib_file,sep='\t',row.names=1,col.names=c("ID","Sequence","Peptide"),header=F)
head(Pep30lib)

# load data and merge with reference
samples=c('KO5', 'WT5') # BAG6 KO and WT


bins=c('HIGH', 'MIDHIGH','MIDLOW','LOW') # each sample sorted into 4 bins

# load other data
for (sample in samples){
    
    print(sample)
    
    for (bin in bins){
        filename=paste(sample,'-',bin,'.count',sep='')
        print(filename)
        x=read.csv(filename,header=F,row.names=1,col.names=c("ID",paste(sample,'-',bin,sep='')),sep='\t')
        Pep30lib = merge(Pep30lib,x,by=0,all=T)
        row.names(Pep30lib) <- Pep30lib$Row.names
        Pep30lib<- Pep30lib[-1]
        print(head(Pep30lib))
    }
}
Pep30lib[is.na(Pep30lib)] <- 0
write.table(Pep30lib,file="Pep30lib-BAG6-KO-MPRA-rep2.txt")

,Sequence,Peptide
,<chr>,<chr>
NM_022844_cds_31_0_chr16_15854397_r(-)|12,CTGCTCAGAAAGTTTGCCACCTCATGGGAATTAATGTGACAGATTTCACCAGATCCATCCTCACTCCTCGTATCAAGGTTGGGCGAGATG,LLRKFATSWELM
NM_001145365_cds_2_0_chr17_47389289_r(-)|30,CACACACAAAAGACATGCCATTTACATGCGAAACCTGTGGAAAATCATTCAAACGCAGTATGTCACTCAAGGTGCACTCCTTGCAGCATT,HTQKTCHLHAKPVENHSNAVCHSRCTPCSI
NM_198150_cds_1_0_chr5_94901702_f(+)|30,GATGGAAGGTTAACATTTCATCCAGGAAGTCAGGTAGTGAAACTTCCTTTTATCAACTTTATGAAGACACGTGGGACTTCCTTTCTGAAT,DGRLTFHPGSQVVKLPFINFMKTRGTSFLN
NM_001282_cds_14_0_chr17_33998773_f(+)|30,GTGGGACAATCCTTCATCCCATCATCGGTGCCTGCAACCTTTGCTCCTTCACCTACACCTGCTGTGGTCAGCAGTGGACTGAATGACCTG,VGQSFIPSSVPATFAPSPTPAVVSSGLNDL
NM_001080851_cds_1_0_chrX_72093384_r(-)|30,GTCCCCAAAGTCTCTGACCAGGCTTTGGTTTCTGCCCACTCAGAGTGGCAGCGGAAGCTGGAGGCCGCTGAGGCTCTGCTGACTCTGAGA,VPKVSDQALVSAHSEWQRKLEAAEALLTLR
NM_019024_cds_17_0_chr2_37268276_r(-)|9,GTTGAAATCGGCTCGAGATGTTGTATCTAGGACTGGTCATTCATTGGCTCTTGGTTGTTTGCATCGTTATGTTGGTGGAATAGGCTCAGG,VEIGSRCCI


[1] "KO5"
[1] "KO5-HIGH.count"
                                                                                                            Sequence
3xFLAG_ACTButr3           GATTATAAAGATCATGACGGAGATTATAAAGACCATGATATTGATTATAAAGATGACGATGATAAGtaaGCGGACTATGACTTAGTTGCG
3xHA                      GGATATCCTTACGATGTTCCAGACTATGCTGGATACCCTTATGATGTCCCAGATTACGCAGGATCATATCCATACGATGTTCCGGATTAC
ACTB(randTerByP)          ATTAATGCCTCTGTGAAAAGGATCCGCATATTAGAGTGTTGTCTGTCTCTATTCCTTATATTTAGCGTACAAGCAGGAATTTCTAGAACT
ACTB(TerByP)              GCGGACTATGACTTAGTTGCGTTACACCCTTTCTTGACAAAACCTAACTTGCGCAGAAAACAAGATGAGATTGGCATGGCTTTATTTGTT
AGO2(randTerByP)_ACTButr3 CTGTCATGCAACAATTTGGCCGACAGGCTAGGGGACGCACAAGCAGTCAGCCGCTTGTCAGGTACTCTACGATCATATGTTTCTTGAGCG
AGO2(TerByP)_ACTButr3     CATGTTTTAGTGTTTAGCGATTGTGTACCGAGTGGGATTCACGAGACCAGCTACACTCAGACCAACAGATGGCCAGCCCTTCCGTGAGCG
                                                 Peptide KO5.HIGH
3xFLAG_ACTButr3                   DYKDHDGDYKDHDIDYKDDDDK       NA
3xHA              

In [132]:
# R script to analyze the BAG6 KO vs WT MPRA data

rm(list = ls())  

rep = 1

d = read.table(paste("Pep30lib-BAG6-KO-MPRA-rep",rep,".txt",sep=''))

colnames(d) <- c("Sequence","Peptide","KO.LOW","KO.MIDLOW","KO.MIDHIGH","KO.HIGH","WT.LOW","WT.MIDLOW","WT.MIDHIGH","WT.HIGH")

d$ID = row.names(d)

# tail length
d$tail_length = apply(as.matrix(d$Peptide),1,nchar)

# only look at 30aa sequences with sufficient coverage 
min_total_count = 3000
total_wt = d$WT.HIGH+d$WT.LOW+d$WT.MIDHIGH+d$WT.MIDLOW
total_ko = d$KO.HIGH+d$KO.LOW+d$KO.MIDHIGH+d$KO.MIDLOW

subset = (total_wt > min_total_count) & (total_ko > min_total_count) & (d$tail_length == 30)
d=d[subset,]
total_wt = total_wt[subset]
total_ko = total_ko[subset]

dim(d)

# expression as slope
pseudo=1
x=1:4
for (i in 1:nrow(d)){
  y = as.numeric(d[i,3:6]+pseudo)
  m = lm(y~x)
  d$expression_ko[i] = m$coefficients[2]/min_total_count*2.5
  d$expression_ko[i] = exp(d$expression_ko[i])/(1+exp(d$expression_ko[i]))
  y = as.numeric(d[i,7:10]+pseudo)
  m = lm(y~x)
  d$expression_wt[i] = m$coefficients[2]/min_total_count*2.5
  d$expression_wt[i] = exp(d$expression_wt[i])/(1+exp(d$expression_wt[i]))
}

prefix='Fig5-'
if (rep != 1){
    prefix='Ext-fig-'
}
               
pdf(paste(prefix,'expression-distribution.pdf',sep=''),width=3.2,height=2.6)
par(mar=c(4,4,1,1))
plot(density(d$expression_ko,from=0,to=1),lwd=2,col='red',xlab='Normalized expression\n',main='',bty='n')
lines(density(d$expression_wt),lwd=2)
legend('topright',c("WT","BAG6 KO"),text.col=c("black","red"),bty='n')
dev.off()


[1] 3661   12

PNG 
  2

In [133]:

# calculate hydrophobicity
library(Peptides)
hydro = hydrophobicity(d$Peptide,"Miyazawa")

# correlation between hydrophobicity and stabilization
pdf(paste(prefix,'hydro-vs-stabilization.pdf',sep=''),width=3.2,height=3.2)
par(mar=c(4,4,1,1))
plot(hydro,d$expression_ko - d$expression_wt,cex=0.3,pch=16,ylim=c(-0.25,0.7),xlab='Hydrophobicity\n',ylab='Stabilization (expression: KO-WT)',bty='l')
cor(hydro,d$expression_ko - d$expression_wt,method='s')
ct=cor.test(hydro,d$expression_ko - d$expression_wt,method='s')
legend('top',bty='n',legend=paste("R = ",format(ct$estimate,digits=3),", P = ",format(ct$p.value,digits=3),sep=''))
#abline(h=0)
dev.off()


[1] 0.3403113

Warning message in cor.test.default(hydro, d$expression_ko - d$expression_wt, method = "s"):
“Cannot compute exact p-value with ties”


PNG 
  2

In [134]:
d3 = data.frame(hydro)
d3$group = 'none'
unstable = d$expression_wt < 0.8
stable = d$expression_wt >= 0.8
stabilized    = d$expression_ko - d$expression_wt > 0
nonstabilized = d$expression_wt - d$expression_ko >= 0
d3$group[unstable & nonstabilized] = '1'
d3$group[unstable & stabilized] = '0'
d3$group[stable ] = '2'

#dev.off()
#par(cex=1,mar=c(6,8,1,1))
#vioplot(hydro~group,data=d3,col=rgb(0.1,0.4,0.7,0.7),horizontal = T,axes=F,xaxt='n',drawRect=T,ylab='',xlab='Hydrophobicity')
#axis(2,c(1,2,3),c(paste('Unstable\nStabilized\nN=',sum(unstable & stabilized),sep=''),
#                  paste('Unstable\nNonstabilized\nN=',sum(unstable & nonstabilized),sep=''),
#                  paste('Stable\nN=',sum(stable),sep='')),las=1)

test = wilcox.test(hydro[unstable & nonstabilized],hydro[unstable & stabilized])
format(test$p.value,digits=1)

test = wilcox.test(hydro[stable],hydro[unstable & nonstabilized])
format(test$p.value,digits=1)

#dev.off()
pdf(paste(prefix,'stable-stabilized-nonstabilized.pdf',sep=''),width=5,height=3.5)
par(cex=1,mar=c(4,4,1,1))
plot(density(hydro[stable]),xlim=c(3.5,8.5),bty='l',main='',xlab='Hydrophobicity\n')
lines(density(hydro[unstable & stabilized]),col='blue')
lines(density(hydro[unstable & nonstabilized]),col='red')
legend('topright',legend=c(paste('Stabilized, N=',sum(unstable & stabilized),sep=''),
                           paste('Nonstabilized, N=',sum(unstable & nonstabilized),sep=''),
                           paste('Stable, N=',sum(stable),sep='') ),
       ,text.col=c('blue','red','black'),box.lwd=0,bty='n')
dev.off()

[1] "2e-18"

[1] "2e-81"

PNG 
  2